In [0]:
display(dbutils.fs.ls("dbfs:/databricks-datasets/power-plant/data/"))

path,name,size,modificationTime
dbfs:/databricks-datasets/power-plant/data/Sheet1.tsv,Sheet1.tsv,308693,1596569238000
dbfs:/databricks-datasets/power-plant/data/Sheet2.tsv,Sheet2.tsv,308693,1596569238000
dbfs:/databricks-datasets/power-plant/data/Sheet3.tsv,Sheet3.tsv,308693,1596569238000
dbfs:/databricks-datasets/power-plant/data/Sheet4.tsv,Sheet4.tsv,308693,1596569238000
dbfs:/databricks-datasets/power-plant/data/Sheet5.tsv,Sheet5.tsv,308693,1596569238000


In [0]:
%sql
CREATE CATALOG IF NOT EXISTS power_plant;
CREATE SCHEMA IF NOT EXISTS power_plant.core;
CREATE VOLUME IF NOT EXISTS power_plant.core.storage;

In [0]:
bronze_checkpoint = "volume://power_plant.core.storage/bronze/checkpoints"
bronze_schema = "volume://power_plant.core.storage/bronze/schemas"
Storage_for_ML_artifacts = "volume://power_plant.core.storage/ml/"

In [0]:
from pyspark.sql.functions import *

bronze_checkpoint = "/Volumes/power_plant/core/storage/bronze/checkpoints"
bronze_schema = "/Volumes/power_plant/core/storage/bronze/schemas"

source_path = "dbfs:/databricks-datasets/power-plant/data/"


df_stream = (spark.readStream.format("cloudFiles")
    .option("cloudFiles.format", "csv")  # Use CSV format
    .option("sep", "\t")  # Tab delimiter for TSV files
    .option("header", "true")
    .option("cloudFiles.inferColumnTypes", "true")
    .option("cloudFiles.schemaLocation", bronze_schema)
    .load(source_path)
)

(df_stream.writeStream.format("delta")
    .option("checkpointLocation", bronze_checkpoint)
    .trigger(once=True)
    .toTable("power_plant.core.bronze_powerplant")
)